In [1]:
import open3d as o3d
import numpy as np
import copy

import time

In [2]:
# transforem the source cloud and then draw the point clouds
def draw_registration_result(source, target, transformation):
    #copy original point clouds
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #set the color
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    #apply transformation
    source_temp.transform(transformation)
    #draw the clouds
    o3d.visualization.draw_geometries([source_temp,target_temp])
    #o3d.visualization.draw_geometries([target])

In [3]:
#down sample the point cloud, recompute new normals, compute fpfh features
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    #use the neighbors recompute the normal
    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    #compute the FPFH features
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh



In [4]:
#load data, down sample and compute fpfh feature
def prepare_dataset(voxel_size):
    #load point clouds
    print(":: Load two point clouds and disturb initial pose.")
    source = o3d.io.read_point_cloud("/home/yunke/3dv_proj/code/objects/0427/cereal_y/cereal_y_stand.pcd")
    target = o3d.io.read_point_cloud("/home/yunke/3dv_proj/code/objects/0427/cereal_y/cereal_y_lie.pcd")
    
    #apply an inital transformation
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    #visualize
    draw_registration_result(source, target, np.identity(4))

    #downsample and compute fpfh feature of the point clouds
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [5]:
#global registration based on RANSAC
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    
    #apply RANSAC algorithms for registration:
    #pick 4 points,
    #pruning with edge length and distance for early stop
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

In [6]:
#fast global registration
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [7]:
#point to plane registration
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.registration.registration_icp(
        source, target, distance_threshold, result_fast.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    return result

In [8]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])


In [92]:
#merge overlapped surface
def merge_registrated(source, target, voxel_size, result_icp):

    #copy point clouds
    source_temp = copy.deepcopy(source)
    source_temp.transform(result_icp.transformation)
    target_temp = copy.deepcopy(target)
    source_temp.normalize_normals()
    target_temp.normalize_normals()
    
    #get points and normals
    src_points = np.asarray(source_temp.points)
    src_normals = np.asarray(source_temp.normals)
    tgt_points = np.asarray(target_temp.points)
    tgt_normals = np.asarray(target_temp.normals)
    num_tgt = tgt_points.shape[0]
    num_tgt_src = tgt_points.shape[0]+src_points.shape[0]
    
    #compute kdtree:
    tgt_src = target_temp+source_temp
    tgt_src_points = np.asarray(tgt_src.points)
    tgt_src_normals = np.asarray(tgt_src.normals)
    tgt_src_colors = np.asarray(tgt_src.colors)
    tgt_src_tree = o3d.geometry.KDTreeFlann(tgt_src)
    
    new_pcd_np = []
    new_color_np = []
    for i in range(num_tgt_src):
        #get neighbors in kdtree
        [k,idx,_] = tgt_src_tree.search_radius_vector_3d(tgt_src.points[i], 0.01)  
        idx = np.asarray(idx)
        #get ones from source or target
        if i<num_tgt:
            near_src = idx[idx>num_tgt-1]
        else:
            near_src = idx[idx<num_tgt]
        #if none, add to the np
        if len(near_src)==0:
            new_pcd_np.append(tgt_src_points[i])
            new_color_np.append(tgt_src_colors[i])
        #else compute nearest one perp to normal drct
        else:
            cur_point = tgt_src_points[i]
            cur_normal = tgt_src_normals[i]
            
            near_src_points = tgt_src_points[near_src]
            near_src_vec = near_src_points-cur_point
            near_src_dist = np.linalg.norm(near_src_vec-np.dot(np.dot(near_src_vec,cur_normal.reshape((3,1))),cur_normal.reshape((1,3))),axis=1)
            nearest_idx = near_src[np.argmin(near_src_dist)]
            #dot the two normals
            nearest_normal = tgt_src_normals[nearest_idx]
            nearest_point = tgt_src_points[nearest_idx]
            dot_normal = np.dot(cur_normal,nearest_normal)
            #if angle small,get one new point
            if dot_normal>0.75 or dot_normal<-0.75:
                # only compute once
                if i<num_tgt:
                    new_pcd_np.append(0.5*(cur_point+nearest_point))
                    new_color_np.append(0.5*(tgt_src_colors[i]+tgt_src_colors[nearest_idx]))
            #else add original point
            else:
                new_pcd_np.append(cur_point)
                new_color_np.append(tgt_src_colors[i])
    
    new_pcd_np = np.asarray(new_pcd_np)
    new_color_np = np.asarray(new_color_np)
    #build point cloud
    final_pcd = o3d.geometry.PointCloud()
    final_pcd.points = o3d.utility.Vector3dVector(new_pcd_np)
    final_pcd.colors = o3d.utility.Vector3dVector(new_color_np)
    final_pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
    #merge
    return final_pcd

In [57]:
voxel_size = 0.007  # means 5cm for the dataset

#down sample, compute fpfh of the data
source, target, source_down, target_down, source_fpfh, target_fpfh = \
        prepare_dataset(voxel_size)


#global registration using ransac
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print("Global registration took %.3f sec.\n" % (time.time() - start))
print(result_ransac)


# #draw the result
draw_registration_result_original_color(source_down, target_down,
                         result_ransac.transformation)

#fast global registration
start = time.time()
result_fast=execute_fast_global_registration(source_down, target_down,
                                             source_fpfh, target_fpfh,
                                             voxel_size)
                         
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
draw_registration_result_original_color(source_down, target_down,
                         result_fast.transformation)
    
#local refinement
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
# print(result_icp)
draw_registration_result_original_color(source, target, result_icp.transformation)


source_temp = copy.deepcopy(source)
target_temp = copy.deepcopy(target)
source_temp.transform(result_icp.transformation)
final_without_refine = target_temp+source_temp
o3d.visualization.draw_geometries([final_without_refine])


:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.007.
:: Estimate normal with search radius 0.014.
:: Compute FPFH feature with search radius 0.035.
:: Downsample with a voxel size 0.007.
:: Estimate normal with search radius 0.014.
:: Compute FPFH feature with search radius 0.035.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.007,
   we use a liberal distance threshold 0.011.
Global registration took 3.342 sec.

registration::RegistrationResult with fitness=8.611017e-01, inlier_rmse=5.126419e-03, and correspondence_set size of 5065
Access transformation to get result.
:: Apply fast global registration with distance threshold 0.004
Fast global registration took 0.092 sec.

registration::RegistrationResult with fitness=2.874872e-01, inlier_rmse=2.649722e-03, and correspondence_set size of 1691
Access transformation to get result.
:: Point-to-plane ICP registration is applied on original point
   clouds t

In [95]:
#compare original and new points of objects
final_down=final_without_refine.voxel_down_sample(voxel_size*0.1)
o3d.visualization.draw_geometries([final_down])
final_pcd = merge_registrated(source, target, voxel_size, result_icp)

o3d.visualization.draw_geometries([final_pcd])
